# Forward Modeling Using Resipy

## Run this first cell to setup your notebook

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
sys.path.append((os.path.relpath('../src'))) # add here the relative path of the API folder
import matplotlib.pyplot as plt
import pandas as pd

import ipywidgets

import numpy as np # numpy for electrode generation
from resipy import Project

#This adjusts the plot settings in the notebook
plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True

plt.rcParams["figure.figsize"] = (25, 8) #Adjust these values to adjust plot sizes. (1st value is horizontal/width, 2nd is vertical/height)

## Create ResiPy Project

Also, start again at this cell to reset your project if you keep running into errors

In [ ]:
k = Project(typ='R2') # create R2 object

## Input Electrode and Profile Geometry Information

Run cell and follow instructions to input data.

***Set electrode spacing of 2 (meters) and 64 electrodes.***

In [ ]:
electrodeSpacing = 
numberOfElectrodes = 


def generateSurfaceElectrodes(resipy_project, electrode_spacing, number_of_electrodes):
  elecDF = pd.DataFrame()

  elec = np.zeros((number_of_electrodes,3))

  elec[:,0] = np.arange(0, number_of_electrodes*electrode_spacing, electrode_spacing)

  elecDF = pd.DataFrame(elec)
  elecDF.columns = ['x','y','z']
  elecDF['remote'] = False
  elecDF['buried'] = False
  elecDF['label'] = elecDF.index.values+1

  resipy_project.setElec(elecDF)
  
  return elecDF
  
generateSurfaceElectrodes(resipy_project=k, electrode_spacing=electrodeSpacing, number_of_electrodes=numberOfElectrodes)
k.elec

# Q1: What is the physical length of the simulated profile in the x-direction that we created above(please include units)?

-----

## Input Background Resistivity Value of Subsurface

Run the cell below and follow the prompts to create a mesh with a background resistivity of 110 ohm-m

In [ ]:
backgroundResistivity = 

k.createMesh(typ='trian', cl_factor = 10, res0=backgroundResistivity) # let's create the mesh based on these electrodes position
k.showMesh()

Typical resistivities of geologic materials

<img src="./resistivity-graph.jpg" alt="Typical Resistivities of Geologic Materials" />

# Q2: Given the background resistivity value you entered and the provided chart, would you expect the subsurface here to be clayey sediment or sandy sediment?

> Note: Use the chart above for your answer to Q2, but note that these values will vary between regions

---

## Input Geometry of Subsurface Target or Layer

Run the cells below and follow the prompts to create an object with the following characteristics:


* The top of the object is 5 meters below the surface
* The bottom of the object is 15 meters below the surface
* Object goes from 60 to 70 meters x-distance
* Resistivity of object is 5 ohm-m

This step addes to the current mesh in your project, so you may need to run the cell(s) above to reset your mesh if you only want a single region.

In [ ]:
# Depths should be negative numbers
topDepthofObject = 
bottomDepthofObject = 

# Locations in meters (positive values)
leftSideOfObject = 
rightSideofObject = 

targetResistivity = 

k.addRegion(np.array([[leftSideOfObject,topDepthofObject],[rightSideofObject,topDepthofObject],[rightSideofObject,bottomDepthofObject],[leftSideOfObject,bottomDepthofObject],[leftSideOfObject,topDepthofObject]]), targetResistivity)
k.showMesh()

## Choose the type of ERT array we will simulate and the expected errors

Run the cell below and follow the prompts to input a measurement error of 10% and set the ERTArray variable equal to 'schlum2' (a type of Schlumberger array).

The available arrays are:
| "Friendly" Name | Name in Resipy|
|-----------------|---------------|
| 'Dipole-Dipole1'| 'dpdp1'     |
| 'Dipole-Dipole2'| 'dpdp2'     |
| 'Wenner (alpha)'|'wenner_alpha'|
| 'Wenner (beta)' |'wenner_beta'|
| 'Wenner (gamma)'|'wenner_gamma'|
|'Schlumberger 1' |'schlum1'|
|'Schlumberger2'  |'schlum2'|
|'Multi-Gradient' |'multigrad'|
|'Custom Sequence'|'custSeq'|


In [ ]:
#Percentage, between 0 and 1
measrument_error_pct =

# ERT array, as string recognized by resipy
ERTArray =

Run the cell below and set the dataLevels variable equal to ***50***

In [ ]:
dataLevels = 

generateSurfaceElectrodes(k, electrodeSpacing, numberOfElectrodes)
k.createMesh(typ='trian', cl_factor = 10, res0=backgroundResistivity) # let's create the mesh based on these electrodes position
k.addRegion(np.array([[leftSideOfObject,topDepthofObject],[rightSideofObject,topDepthofObject],[rightSideofObject,bottomDepthofObject],[leftSideOfObject,bottomDepthofObject],[leftSideOfObject,topDepthofObject]]), targetResistivity)

if ERTArray in ['dpdp1', 'dpdp2', 'schlum1','schlum2']:
  k.createSequence([(ERTArray, 1, dataLevels)])
else:
  wennerList = []
  for w in range(1,dataLevels):
    wennerList.append((ERTArray,w))
  k.createSequence(wennerList)

In [ ]:
k.forward(noise=measrument_error_pct, iplot=True) 

# Q3: Include a copy of this initial apparent resistivity pseudosection plot. (This should look like a plot with colored points)

---

## Run an inversion on the pseudosection generated from our forward model

In [ ]:
k.invert()

# Q4: After running your initial inversion, what is the final RMS misfit (the last value in the output table)? Since the data we read in for the inversion was derived from a forward model, would you expect the value to be zero? Explain why or why not.

---

## Modeling results

This first chart shows the original model we created, color-coded now by resistivity

In [ ]:
k.showResults(index=0, attr='Resistivity(ohm.m)', color_map='cividis', sens=False, contour=False, clipCorners=False)

This second chart shows the results of our inversion on the pseudosection calculated using our forward model. 

In [ ]:
k.showResults(index=1, attr='Resistivity(ohm.m)', color_map='cividis', sens=False, contour=True, clipCorners=False)

# Q5: Include a copy of the two plots from the Modeling Results section. How do they compare and what might explain any differences?

---

## Change array

Now, let's see if our model changes at all if we use a different array. 

Set ERTArray equal to 'dpdp2' (i.e., Dipole-Dipole2 array)

The available arrays are:
| "Friendly" Name | Name in Resipy|
|-----------------|---------------|
| 'Dipole-Dipole1'| 'dpdp1'     |
| 'Dipole-Dipole2'| 'dpdp2'     |
| 'Wenner (alpha)'|'wenner_alpha'|
| 'Wenner (beta)' |'wenner_beta'|
| 'Wenner (gamma)'|'wenner_gamma'|
|'Schlumberger 1' |'schlum1'|
|'Schlumberger2'  |'schlum2'|
|'Multi-Gradient' |'multigrad'|
|'Custom Sequence'|'custSeq'|


> Note that we are not changing any other parameters aside from the array type.

In [ ]:
ERTArray =

Now, use the cell below to run a forward model and plot the pseudosection

In [ ]:
generateSurfaceElectrodes(k, electrodeSpacing, numberOfElectrodes)
k.createMesh(typ='trian', cl_factor=10, res0=backgroundResistivity) # let's create the mesh based on these electrodes position
k.addRegion(np.array([[leftSideOfObject,topDepthofObject],[rightSideofObject,topDepthofObject],[rightSideofObject,bottomDepthofObject],[leftSideOfObject,bottomDepthofObject],[leftSideOfObject,topDepthofObject]]), targetResistivity)

if ERTArray in ['dpdp1', 'dpdp2', 'schlum1','schlum2']:
  k.createSequence([(ERTArray, 1, dataLevels)])
else:
  wennerList = []
  for w in range(1,dataLevels):
    wennerList.append((ERTArray,w))
  k.createSequence(wennerList)

k.forward(iplot=True) 

And run an inversion using the new array.

In [ ]:
k.invert()
k.showResults(index=1, attr='Resistivity(ohm.m)', color_map='cividis', sens=False, contour=True, clipCorners=False)

# Q6: In 1-4 sentences, comment on the differences and/or similarities between the pseudosections and the inversions between the two model runs we have done now (the first using a Schlumberger array and the second using a dipole-dipole array). 

Which do you like better? Which has a better misfit between the pseudosection and forward model?

> Note that in field situations, often the dipole-dipole array can measure 4-8x as fast as the Schlumberger array. Does this change your answer to Q6?

---

## Add layer

Now, create a geologic layer with the following parameters:


*   Top Depth: -5
*   Bottom Depth: -10
*   Left: 0
*   Right: 75
*   Resistivity: 200



In [ ]:
topDepthofObject =
bottomDepthofObject =
leftSideOfObject =
rightSideofObject =
targetResistivity =

generateSurfaceElectrodes(k, electrodeSpacing,numberOfElectrodes)
k.createMesh(typ='trian', cl_factor=10, res0=backgroundResistivity) # let's create the mesh based on these electrodes position
k.addRegion(np.array([[leftSideOfObject,topDepthofObject],[rightSideofObject,topDepthofObject],[rightSideofObject,bottomDepthofObject],[leftSideOfObject,bottomDepthofObject],[leftSideOfObject,topDepthofObject]]), targetResistivity)
k.showMesh()

In [ ]:
k.forward(noise=measrument_error_pct,iplot=True) 

In [ ]:
k.invert()
k.showResults(index=1, attr='Resistivity(ohm.m)', color_map='cividis', sens=False, contour=True, clipCorners=True)

# Q7:  The forward model and inversion above represent a layered subsurface. Describe the following:
a) What your forward model and inversion outputs represent

b) How your inversion output compares to the geological model we constructed
* We constructed a geologic model when we specified the top/bottom/left/right of our subsurface target

c) How this compares with the first model we processed.


> Note that using the default inversion parameters as we did in this section of the exercise may not yield highly targeted results for this or other ERT data (also, using a more sophisticated software like Res2DInv might yield better inversions).

---

## Add downhole electrodes

First, let's regenerate our dataframe with the surface electrodes

In [ ]:
generateSurfaceElectrodes(k, electrodeSpacing, numberOfElectrodes)
k.elec

Now, let's add our downhole electrodes (these are electrodes that would be put in a screened, pvc well)

In [ ]:
generateSurfaceElectrodes(k, electrodeSpacing, numberOfElectrodes) #Just in case, to make sure we are starting with the rightSideofObject electrodes

newElecDF = k.elec.copy()

for x in [41, 63]:
  depth = -4 # We do not start at 0, since downhole electrodes need to be in water
  for i in range(64, 96):
    itemList = []
    itemList.append(x) #X-location of downhole electrode
    itemList.append(0.0) #Y location
    itemList.append(depth) #z location/Depth of downhole electrode
    itemList.append(False) # Not a remote electrode (used only in pole arrays)
    itemList.append(True) # It is a a buried electrode
    itemList.append(i+1) #Add electrodeLabel
    itemDF = pd.DataFrame(itemList).transpose() #Turn list into dataframe for appending
    depth -= 1.5 # Increment the depth so next electrode is 5 meters deeper
    itemDF.columns=['x','y','z','remote','buried','label'] # add column names
    itemDF.index=[i] #add index value
    
    if i < 80:
      newElecDF = pd.concat([newElecDF, itemDF]) # Add downhole electrodes to existing electrode dataframe

k.setElec(newElecDF) #Set new electrode layout using new dataframe
k.elec

In [ ]:
k.createMesh(typ='trian', cl=5, res0=backgroundResistivity) # let's create the mesh based on these electrodes position
k.addRegion(np.array([[leftSideOfObject,topDepthofObject],[rightSideofObject,topDepthofObject],[rightSideofObject,bottomDepthofObject],[leftSideOfObject,bottomDepthofObject],[leftSideOfObject,topDepthofObject]]), targetResistivity)
k.showMesh()

In [ ]:
k.forward(noise=measrument_error_pct, iplot=True)

In [ ]:
k.invert()
k.showResults(index=1, attr='Resistivity(ohm.m)', color_map='cividis', sens=False, contour=True, clipCorners=True, zlim=[-25, 0])

# Q8: Describe any differences between the inverted layered model before and after we added downhole electrodes.  Include the plot from the final code cell.

# Q9: What is the biggest insight you took away from this exercise?